In [36]:
%pylab inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sns
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

Populating the interactive namespace from numpy and matplotlib


In [37]:
train_data = pd.read_csv('../data/scaled_train.csv', index_col='PassengerId')
train_y = train_data['Survived']
train_X = train_data.drop(['Survived'], axis=1)

In [4]:
cv = StratifiedKFold(train_data["Survived"], n_folds=3, shuffle=True, random_state=1)

alg_sgd = SGDClassifier(random_state=1)
scores = cross_val_score(alg_sgd, train_X, train_y, cv=cv, n_jobs=-1)
print("Accuracy (sgd): {}/{}".format(scores.mean(), scores.std()))

#alg_sgd = SGDClassifier(random_state=1, penalty='l2')
#alg_sgd.fit(train_X, train_y)
#alg_sgd.score(train_X, train_y)

Accuracy (sgd): 0.765432098765/0.0249451299337


In [38]:
cv = StratifiedKFold(train_data["Survived"], n_folds=5, shuffle=True, random_state=1)
alg_frst = RandomForestClassifier(random_state=1, n_estimators=5000, min_samples_split=8, min_samples_leaf=2)
scores = cross_val_score(alg_frst, train_X, train_y, cv=cv, n_jobs=-1)
print("Accuracy (random forest): {}/{}".format(scores.mean(), scores.std()))

Accuracy (random forest): 0.831678456564/0.0181948704487


In [40]:
cv = StratifiedKFold(train_data["Survived"], n_folds=5, shuffle=True, random_state=1)

alg_frst_model = RandomForestClassifier(random_state=1)
alg_frst_params = [{
    "n_estimators": [350, 400, 450, 500],
    "min_samples_split": [6, 8, 10],
    "min_samples_leaf": [1, 2, 4]
}]
alg_frst_grid = GridSearchCV(alg_frst_model, alg_frst_params, cv=cv, refit=True, verbose=1, n_jobs=4)
alg_frst_grid.fit(train_X, train_y)
alg_frst_best = alg_frst_grid.best_estimator_
print("Accuracy (random forest auto): {} with params {}"
      .format(alg_frst_grid.best_score_, alg_frst_grid.best_params_))

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   31.8s
[Parallel(n_jobs=4)]: Done 180 out of 180 | elapsed:  2.6min finished


Accuracy (random forest auto): 0.832772166105 with params {'min_samples_split': 8, 'n_estimators': 400, 'min_samples_leaf': 2}


In [31]:
alg_nbs = GaussianNB()
scores = cross_val_score(alg_nbs, train_X, train_y, cv=cv, n_jobs=-1)
print("Accuracy (naive bayes): {}/{}".format(scores.mean(), scores.std()))

Accuracy (naive bayes): 0.789001122334/0.0111105442611


In [ ]:
range(1, 10, 2)

In [38]:
for f in range(11, 12, 1):
    cv = StratifiedKFold(train_data["Survived"], n_folds=f, shuffle=True, random_state=1)

    ald_xgb_model = xgb.XGBClassifier()
    ald_xgb_params = [
        {
         "n_estimators": [700],
         "max_depth": [12],
         "learning_rate": [0.005]
        }
    ]
    alg_xgb_grid = GridSearchCV(ald_xgb_model, ald_xgb_params, cv=cv, refit=True, verbose=1, n_jobs=4)
    alg_xgb_grid.fit(train_X, train_y)
    alg_xgb_best = alg_xgb_grid.best_estimator_
    print("Accuracy (xgboost auto): {} with params {}"
          .format(alg_xgb_grid.best_score_, alg_xgb_grid.best_params_)
         )
    print f


Fitting 11 folds for each of 1 candidates, totalling 11 fits


[Parallel(n_jobs=4)]: Done  11 out of  11 | elapsed:    3.6s finished


Accuracy (xgboost auto): 0.843995510662 with params {'n_estimators': 700, 'learning_rate': 0.005, 'max_depth': 12}
11


In [ ]:
0.84399